In [33]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

file = 'example.mp4'
cap = cv2.VideoCapture(file)

frames = []
gray_frames = []

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret:
        frames.append(frame)
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frames.append(gray_frame)
    else:
        break
        
    key = cv2.waitKeyEx(1)
    
    if key == 27:
        break
    

cap.release()
cv2.destroyAllWindows()

In [39]:
gray_frames_array = np.stack(gray_frames)

In [40]:
background = np.median(gray_frames_array, axis=0)
background = background.astype('uint8')

In [41]:
cap = cv2.VideoCapture(file)
# cv2.namedWindow("frame", cv2.WINDOW_NORMAL) 

drone_frames = []
masks = []

while(cap.isOpened()):
    ret, frame = cap.read()

    if ret:
        frames.append(frame)
        
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray_frames.append(gray_frame)

        dframe = cv2.absdiff(gray_frame, background)
        th, mask = cv2.threshold(dframe, 10, 255, cv2.THRESH_BINARY)
        masks.append(mask)
        
        frame = cv2.bitwise_and(frame, frame, mask=mask)
        drone_frames.append(frame)
        
        # cv2.imshow('frame', frame)  
    else:
        break
    key = cv2.waitKeyEx(1)
    
    if key == 27:
        break
    

cap.release()
cv2.destroyAllWindows()

In [42]:
len(drone_frames)

222

In [48]:
image = frames[0]
start_index = 0
end_index = len(drone_frames)-1
for i in reversed(range(start_index,end_index,2)):
    mask = cv2.bitwise_not(masks[i])
    image = cv2.bitwise_and(image, image, mask=mask)
    image = cv2.add(image, drone_frames[i])
#     image = cv2.addWeighted(image, 1,drone_frames[i],0.5,0)

In [49]:
cv2.namedWindow("frame", cv2.WINDOW_NORMAL)

while True:
    cv2.imshow('frame', image)
    
    key = cv2.waitKeyEx(1)
    
    # Press esc to exit
    if key == 27:
        break
cv2.destroyAllWindows()

In [50]:
cv2.imwrite('composite.png', image, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True